In [ ]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types

## Question 1. Install Spark and PySpark
Answer:3.0.3

In [4]:
pyspark.__version__

'3.0.3'

In [5]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [ ]:
!curl https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv -o fhvhv_tripdata_2021-02.csv

In [6]:
df = spark.read.option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [7]:
# df.head(5)
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,StringType,true),StructField(DOLocationID,StringType,true),StructField(SR_Flag,StringType,true)))

In [8]:
df_pandas = pd.read_csv('fhvhv_tripdata_2021-02.csv', nrows=1000)

In [ ]:
df_pandas.dtypes

In [9]:
# spark.createDataFrame(df_pandas).show()
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [10]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType() ,True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [ ]:
schema

In [11]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [ ]:
# df.show()
# df.schema
df.printSchema()

In [13]:
df.repartition(24)

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [14]:
df.write.parquet('06_homework_week_5', mode='overwrite')

## Question 2. HVFHW February 2021
What's the size of the folder with results (in MB)?
Answer: 150M

In [15]:
!dir 06_homework_week_5

 Volume in drive L is H:\Data-Python
 Volume Serial Number is 0464-DD1C

 Directory of L:\python-virtual-environments\de\06_homework_week_5

27/02/2022  11:10    <DIR>          .
27/02/2022  11:10    <DIR>          ..
27/02/2022  11:10           158,208 .part-00000-8621df0e-097d-41d1-b49a-6b15f5e5f0ea-c000.snappy.parquet.crc
27/02/2022  11:10           143,752 .part-00001-8621df0e-097d-41d1-b49a-6b15f5e5f0ea-c000.snappy.parquet.crc
27/02/2022  11:10           147,408 .part-00002-8621df0e-097d-41d1-b49a-6b15f5e5f0ea-c000.snappy.parquet.crc
27/02/2022  11:10           143,320 .part-00003-8621df0e-097d-41d1-b49a-6b15f5e5f0ea-c000.snappy.parquet.crc
27/02/2022  11:10           147,768 .part-00004-8621df0e-097d-41d1-b49a-6b15f5e5f0ea-c000.snappy.parquet.crc
27/02/2022  11:10           143,496 .part-00005-8621df0e-097d-41d1-b49a-6b15f5e5f0ea-c000.snappy.parquet.crc
27/02/2022  11:10           147,776 .part-00006-8621df0e-097d-41d1-b49a-6b15f5e5f0ea-c000.snappy.parquet.crc
27/02/2022  11:10  

In [17]:
df.registerTempTable('trips')

In [18]:
spark.sql("SELECT count(*) from trips").show()

+--------+
|count(1)|
+--------+
|11613942|
+--------+



In [ ]:
df.printSchema()

## Question 3. Count records
-- How many taxi trips were there on February 15?
-- Consider only trips that started on February 15.
Ans: 367170

In [ ]:
spark.sql("""SELECT date(pickup_datetime) as pickup_date, count(*) from trips 
          group by pickup_date order by pickup_date """).show(50)

## Question 4. Longest trip for each day

Now calculate the duration for each trip.
Trip starting on which day was the longest? 
Ans: 11/02/2021 - 

In [ ]:
# get top length in seconds 
spark.sql("""SELECT bigint(dropoff_datetime)-bigint(pickup_datetime) as trip_len_seconds, pickup_datetime from trips  
          order by 1 desc LIMIT 5 """).show()

## Question 5. Most frequent `dispatching_base_num`

Now find the most frequently occurring `dispatching_base_num` 
in this dataset.

How many stages this spark job has?

> Note: the answer may depend on how you write the query,
> so there are multiple correct answers. 
> Select the one you have.
Ans: 1741
Ans: 3 Stages

In [ ]:
spark.sql("""SELECT dispatching_base_num, count(*) from trips  
          group by dispatching_base_num order by dispatching_base_num desc LIMIT 1 """).show()

## Question 6. Most common locations pair

Find the most common pickup-dropoff pair. 
For example:
"Jamaica Bay / Clinton East"
Enter two zone names separated by a slash
If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East". 
Ans: East New York/East New York (45041 records)

## Bonus question. Join type

(not graded) 
For finding the answer to Q6, you'll need to perform a join.
What type of join is it?
And how many stages your spark job has?
Ans: Outer
Ans: 2 Stages

In [ ]:
df_zones = pd.read_csv('taxi_zone_lookup.csv')

In [ ]:
# df_zones.dtypes
df_zones

In [ ]:
schema = types.StructType([
    types.StructField('LocationID', types.IntegerType() ,True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True)
])

In [ ]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('taxi_zone_lookup.csv')

In [ ]:
df_zones.dtypes

In [ ]:
df_zones.registerTempTable('zones')

In [ ]:
spark.sql("""SELECT * from zones order by LocationID""").show()

In [ ]:
from pyspark.sql.functions import concat_ws

In [ ]:
spark.sql("""select concat_ws('/',coalesce(pickup_zone.Zone,'Unknown'), coalesce(dropoff_zone.Zone,'Unknown')) as pickupdropoff,
    count(*)
    from trips
    left join zones as pickup_zone on pickup_zone.LocationID = trips.PULocationID 
    left join zones as dropoff_zone on dropoff_zone.LocationId = trips.DOLocationID
    group by 1
    order by 2 desc
    """).show(truncate=False)


In [ ]:
# df.show()
df.dtypes
# df_zones.dtypes